In [1]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import os
import time
import google.generativeai as genai

genai.configure(api_key="GOOGLE_API_KEY")

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")

# Create the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

# TODO Make these files available on the local file system
# You may need to update the file paths
cwd = os.getcwd()
archivos = os.path.join(cwd, "files")
files = []
for file in os.listdir(archivos):
  try:
    if file.endswith(".pdf"):
      files.append(upload_to_gemini(os.path.join(archivos, file), mime_type="application/pdf"))
    if file.endswith(".txt"):
      files.append(upload_to_gemini(os.path.join(archivos, file), mime_type="text/plain"))
  except Exception as e:
    print(f"Error uploading file {file}: {e}")

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)
prompt_inicial = "Eres un asistente experto en normativa gubernamental de la provincia de tucumán, en base a estos archivos que tienen relación al gobierno de la provincia de tucumán y su administración financiera, contesta mis preguntas intentando ajustarte lo máximo posible al contenido de los mismos"
files.append(prompt_inicial)
chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": files,
    },
    {
      "role": "model",
      "parts": [
        "¡Claro que sí! Estoy listo para responder a tus preguntas sobre el gobierno de la provincia de Tucumán y su administración financiera, basándome en los archivos que me has proporcionado. \n\nPor favor, hazme tus preguntas. 😊 \n",
      ],
    },
    ]
)



Uploaded file 'Cicular Conjunta Nº1-2022.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/5p2zuc05lce6
Uploaded file 'circular 2022_2 tgp_cgp.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/tfsya8biwbi8
Uploaded file 'Circular TGP N 1 - 2023.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/6ip0a7c9v3vt
Uploaded file 'Circular TGP N 1 - Acreditaciones en cuenta SGP - Ret. Impositivas.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/bnfygu2zrhk4
Uploaded file 'Circular TGP Nº 3 -2022.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/tf62nxznqj70
Uploaded file 'COMUNICADO - 2021 - BANCA INTERNET.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/sweglxvgl0dv
Uploaded file 'D 1080_3 ME.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/9ie7e2p6mqug
Uploaded file 'D-2-3-ME-02012008.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/9m8bt1j4di6e
Uploaded file 'D-941-3-ME-16042014

In [2]:
from IPython.display import Markdown, display

def responder(prompt):
    response = chat_session.send_message(prompt)
    display(Markdown(response.text))

In [4]:
responder("que me puedes decir de la circular conjunta nro 1 del 2022")

La Circular Conjunta Nro. 1 del 2022, emitida por la Tesorería General de la Provincia de Tucumán, se refiere a la **obligación de presentar el Certificado de Cumplimiento Fiscal**, según lo establecido en el artículo Nro. 1 del Decreto Nro. 1531/MH del 2 de julio de 1997. 

En resumen, la circular indica que:

* **Todos los organismos, incluyendo los centralizados, descentralizados y autárquicos**, deben presentar el Certificado de Cumplimiento Fiscal.
* La presentación del Certificado debe realizarse **de acuerdo a lo establecido en el artículo Nro. 1 del Decreto Nro. 1531/MH**, que se menciona en la circular. 

La circular también menciona que la normativa **quedará debidamente notificada a partir de la publicación de la presente, en la página web oficial de la TGP**.

¿Te gustaría que buscara información adicional sobre el Decreto Nro. 1531/MH o sobre el Certificado de Cumplimiento Fiscal? 


In [3]:
responder("cuales son las potestades de la dirección de presupuesto")

La Dirección Provincial de Presupuesto, según la Ley de Administración Financiera de la Provincia de Tucumán (Ley 6.970), tiene las siguientes potestades: 

* **Participar en la formulación de los aspectos presupuestarios de la política financiera que, para el sector público provincial, elabore el órgano coordinador de los sistemas de administración financiera.** Esto implica que la Dirección de Presupuesto tiene que estar involucrada en la planificación financiera del sector público provincial. 
* **Proponer al órgano coordinador de los sistemas de administración financiera los lineamientos para la elaboración de los presupuestos del sector público provincial.** Es decir, la Dirección de Presupuesto debe proponer las directrices generales para la elaboración de los presupuestos. 
* **Dictar las normas técnicas para la formulación, programación de la ejecución, modificaciones y evaluación de los presupuestos de la administración provincial.** Esto significa que la Dirección de Presupuesto es la responsable de establecer las reglas y procedimientos para la elaboración, ejecución y seguimiento de los presupuestos. 
* **Dictar las normas técnicas para la formulación y evaluación de los presupuestos de las empresas y sociedades del Estado.**  La Dirección de Presupuesto también tiene que establecer las normas para la elaboración de los presupuestos de las empresas y sociedades que son propiedad del Estado. 
* **Analizar los anteproyectos de presupuesto de los organismos que integran la administración provincial y proponer los ajustes que considere necesarios.** La Dirección de Presupuesto tiene que revisar los presupuestos que proponen los organismos del gobierno provincial y sugerir cambios si lo considera necesario. 
* **Analizar los proyectos de presupuesto de las empresas y sociedades del Estado y presentar los respectivos informes a consideración del Poder Ejecutivo provincial.** La Dirección de Presupuesto debe analizar los presupuestos de las empresas y sociedades del Estado y luego presentar su análisis al Poder Ejecutivo. 
* **Preparar el proyecto de ley de presupuesto general y fundamentar su contenido.** La Dirección de Presupuesto tiene que elaborar el proyecto de ley de presupuesto y explicar los fundamentos de su contenido. 
* **Aprobar, juntamente con la Tesorería General de la Provincia, la programación de la ejecución del presupuesto de la administración provincial preparada por las jurisdicciones y entidades que la componen.** La Dirección de Presupuesto debe aprobar, junto con la Tesorería General, la planificación de la ejecución del presupuesto. 
* **Asesorar, en materia presupuestaria, a todos los organismos del sector público provincial regidos por esta ley y difundir los criterios básicos para un sistema presupuestario compatible a nivel de las municipalidades.** La Dirección de Presupuesto tiene que asesorar a los organismos del gobierno provincial en temas presupuestarios y difundir las mejores prácticas para la elaboración de presupuestos. 
* **Coordinar los procesos de ejecución presupuestaria de la administración provincial e intervenir en los ajustes y modificaciones a los presupuestos, de acuerdo a las atribuciones que le fije la reglamentación.** La Dirección de Presupuesto debe coordinar la ejecución de los presupuestos y participar en los cambios que se realicen a los mismos. 
* **Evaluar la ejecución de los presupuestos, aplicando las normas y criterios establecidos por esta ley, su reglamentación y las normas técnicas respectivas.** La Dirección de Presupuesto debe evaluar la ejecución de los presupuestos y asegurarse de que se cumplan las normas y procedimientos establecidos. 
* **Las demás que le confiera la presente ley y su reglamento.** La Dirección de Presupuesto también tiene las competencias que se establezcan en la ley y su reglamento. 

Espero que esta información te sea útil. ¡No dudes en hacerme más preguntas! 


In [4]:
responder("en que articulo estás eso?")

La información que te he dado sobre las potestades de la Dirección Provincial de Presupuesto se encuentra en el **Artículo 16** de la Ley de Administración Financiera de la Provincia de Tucumán (Ley 6.970). 

En este artículo, se detallan las funciones y competencias de la Dirección Provincial de Presupuesto como órgano rector del sistema presupuestario. 
